In [3]:
import numpy as np
import simtk.openmm as mm
import simtk.openmm.app as app
from simtk.openmm import unit
from simtk.openmm.openmm import NonbondedForce
import os
from openmmtools.testsystems import AlanineDipeptideExplicit
import sys

In [20]:
def percentageString(progress, total, divisions):
    length=total/divisions
    stage=int(progress//length)
    token=">"
    if divisions==stage:
        token="X"
    if divisions-stage<0:
        return "Overcompletion!"
    return "["+"="*stage+token+" "*(divisions-stage)+"]"

In [21]:
name="alad-015M-15A"
run="NOCHARMM-NVT"
resultsdir="results/"+"{}-run-{}".format(name,run)
if not os.path.exists(resultsdir):
    os.mkdir(resultsdir)
    print("done")

In [22]:
constraints = app.HBonds
nonbonded_cutoff = 1.0*unit.nanometer
switch_distance = 0.8*unit.nanometer
nonbonded_method = app.PME
implicit_solvent = False
solvated = True
hydrogen_mass = None

In [23]:
friction = 1.0 / unit.picoseconds
pressure = 1.0 * unit.atmosphere
initial_temperature = 2*unit.kelvin
final_temperature = 298*unit.kelvin
timestep = 2.0 * unit.femtosecond

In [24]:
create_system_kwargs = dict(
    removeCMMotion=True,
    nonbondedMethod=nonbonded_method,
    nonbondedCutoff=nonbonded_cutoff,
    switchDistance=switch_distance,
    constraints=constraints,
    hydrogenMass=hydrogen_mass,
    rigidWater=True,
)

In [25]:
alanine=AlanineDipeptideExplicit(**create_system_kwargs)

In [26]:
system=alanine.system

In [27]:
positions=alanine.positions
topology=alanine.topology

In [28]:
integrator = mm.LangevinIntegrator(initial_temperature , friction , timestep)
integrator.setConstraintTolerance(1e-7)

In [29]:
platform = mm.Platform.getPlatformByName("OpenCL")
platform_properties = {"DeviceIndex": "0", "Precision": "mixed"}

In [30]:
simulation = mm.app.Simulation(topology, system, integrator, platform, platform_properties)
simulation.context.setPositions(positions)

In [31]:
simulation.minimizeEnergy()
simulation.reporters.append(app.DCDReporter(resultsdir+'/NVT-equilibration_{}.dcd'.format(name), 1000))
simulation.reporters.append(app.StateDataReporter(resultsdir+'/{}_scalars.csv'.format(name), 
    1000, time=True, potentialEnergy=True, totalEnergy=True,
    temperature=True, speed=True, elapsedTime=True, volume=True))


In [32]:
n_temp_steps = 100
temperature = initial_temperature
dtemperature = (final_temperature.value_in_unit(unit.kelvin)-initial_temperature.value_in_unit(unit.kelvin)) / n_temp_steps
init_temp_value = initial_temperature.value_in_unit(unit.kelvin)
final_temp_value = final_temperature.value_in_unit(unit.kelvin)

In [33]:
step_size = 100 #200ps simulation
print(f"NVT equilibration will last: {timestep.in_units_of(unit.picosecond)*step_size*n_temp_steps}")
# Total num of steps = step_size * n_temp_steps
for temperature in np.linspace(init_temp_value,final_temp_value,n_temp_steps):
    integrator.setTemperature( temperature*unit.kelvin )
    simulation.step(step_size)
    percentageBar=percentageString(temperature, final_temp_value, 50)
    print("\rT: {:.2f} , {:.3} % {}".format(temperature, temperature/final_temp_value*100, percentageBar), end="")

NVT equilibration will last: 20.0 ps
T: 298.00 , 1e+02 % [==================================================X]

In [38]:
crd = simulation.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions()
app.pdbfile.PDBFile.writeFile(topology, crd, open(resultsdir+'/equilibrated-NVT_{}.pdb'.format(name), 'w'))
box = np.array(simulation.context.getState().getPeriodicBoxVectors().value_in_unit(unit.nanometer))
a,b,c = box[0,0],box[1,1],box[2,2]
np.savetxt(resultsdir+'/NVT-box-lengths_{}.txt'.format(name),np.array((a,b,c)))